In [1]:
import os
import resppol
import resppol.resppol

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
from importlib import reload
reload(logging)

logger = logging.getLogger()
logging.basicConfig(
    format='%(asctime)s %(message)s',
    datefmt='%Y-%m-%d %I:%M:%S %p',
    level=logging.DEBUG
)

In [4]:
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
def loading_trainingset(molecule_number, tf=True,\
                        ROOT_DIR_PATH=ROOT_DIR_PATH):
    # set the unit
    from pint import UnitRegistry
    ureg = UnitRegistry()
    Q_ = ureg.Quantity
    ureg.define('bohr = 0.52917721067 * angstrom')
    ROOT_DIR_PATH = ROOT_DIR_PATH
    all_directions=[['X+',[-0.01,0.0,0.0]],['X-',[0.01,0.0,0.0]],\
                   ['Y+',[0.0,-0.01,0.0]],['Y-',[0.0,0.01,0.0]],\
                   ['Z+',[0.0,0.0,-0.01]],['Z-',[0.0,0.0,0.01]]]
    # set the basic info for traininga_set
    training_set = resppol.resppol.TrainingSet(mode='q_alpha',SCF= False, thole = False, tf=tf,\
                                           FF='resppol/resppol/data/test_data/BCCelementPOL.offxml')
    # load molecules to training set
    
    
    for i in range(1, molecule_number+1):
        mol_path = os.path.join(ROOT_DIR_PATH, f"molecule{i}")
        molecule_conf = os.path.join(mol_path, "conf0", "mp2_0.mol2")
        training_set.add_molecule(molecule_conf)
        for dir in os.listdir(mol_path):
            if "conf" in dir:
                conf_path = os.path.join(mol_path, dir)
                mp2_path = os.path.join(conf_path, f"mp2_{dir.split('conf')[1]}.mol2")
                esp_path = os.path.join(conf_path, f"molecule{dir.split('conf')[1]}.gesp")
                training_set.molecules[i-1].add_conformer_from_mol2(mp2_path)
                training_set.molecules[i-1].conformers[-1].add_baseESP(esp_path)
                for direction, vector in all_directions:
                    esp_file = os.path.join(conf_path, f'molecule_{direction}.gesp')
                    training_set.molecules[i-1].conformers[-1].add_polESP(esp_file, e_field=Q_(vector, 'elementary_charge / bohr / bohr'))
    # print out useful information about the size of training set
    mol_nu = 0
    conf_nu = 0
    for molecule in training_set.molecules:
        mol_nu += 1 
        for conf in molecule.conformers: 
            conf_nu += 1
    print("There are", mol_nu, "molecules,", "and", conf_nu, "conformers in the training set.")
    return training_set

# Small test with one molecule

In [6]:
training_set = loading_trainingset(22)
training_set.optimize_charges_alpha()

There are 1 molecules, and 5 conformers in the training set.


2020-09-30 01:53:32 PM I do not have AM1-type charges
2020-09-30 01:53:32 PM I do not have AM1-type charges
2020-09-30 01:53:32 PM I do not have AM1-type charges
2020-09-30 01:53:32 PM I do not have AM1-type charges
2020-09-30 01:53:32 PM I do not have AM1-type charges
2020-09-30 01:53:33 PM Solving matrix with TF took 284.80 ms.
2020-09-30 01:53:33 PM Finished updating optimization step: 1. 

2020-09-30 01:53:34 PM Solving matrix with TF took 0.69 ms.
2020-09-30 01:53:34 PM Finished updating optimization step: 2. 

2020-09-30 01:53:35 PM Solving matrix with TF took 0.69 ms.
2020-09-30 01:53:35 PM Finished updating optimization step: 3. 

2020-09-30 01:53:36 PM Solving matrix with TF took 0.70 ms.
2020-09-30 01:53:36 PM Finished updating optimization step: 4. 

2020-09-30 01:53:36 PM Solving matrix with TF took 0.69 ms.
2020-09-30 01:53:36 PM Finished updating optimization step: 5. 

2020-09-30 01:53:37 PM Solving matrix with TF took 0.69 ms.
2020-09-30 01:53:37 PM Finished updating op